<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/multi_agent_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparation

In [ ]:
!pip install metagpt==0.5.2

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["OPENAI_API_MODEL"] = "gpt-4-1106-preview"

import re
import asyncio
from metagpt.actions import Action, BossRequirement
from metagpt.roles import Role
from metagpt.team import Team
from metagpt.schema import Message
from metagpt.logs import logger

def parse_code(rsp):
    pattern = r'```python(.*)```'
    match = re.search(pattern, rsp, re.DOTALL)
    code_text = match.group(1) if match else rsp
    return code_text

2023-11-17 03:06:12.082 | INFO     | metagpt.const:get_project_root:27 - PROJECT_ROOT set to current working directory: /content
2023-11-17 03:06:12.645 | INFO     | metagpt.config:__init__:44 - Config loading done.


### Define Action and Role
Following the same process as [Agent101](https://colab.research.google.com/drive/1SF3bJiDjKw6Xwnz2Rf0j8Hc0U4KsSB2L#scrollTo=TJX9A8lh3FbS), we can define three `Role`s with their respective `Action`s:
- A `SimpleCoder` with a `SimpleWriteCode` action, taking instruction from the user and writing the main code
- A `SimpleTester` with a `SimpleWriteTest` action, taking the main code from `SimpleWriteCode` output and providing a test suite for it
- A `SimpleReviewer` with a `SimpleWriteReview` action, reviewing the test cases from `SimpleWriteTest` output and check their coverage and quality

By giving the outline above, we actually make our SOP clear. We will talk about how to set up the `Role` according to it shortly.

#### Define Action
We list the three `Action`s.

In [ ]:
class SimpleWriteCode(Action):

    PROMPT_TEMPLATE = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    def __init__(self, name="SimpleWriteCode", context=None, llm=None):
        super().__init__(name, context, llm)

    async def run(self, instruction: str):

        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

class SimpleWriteTest(Action):

    PROMPT_TEMPLATE = """
    Context: {context}
    Write {k} unit tests using pytest for the given function, assuming you have imported it.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    def __init__(self, name="SimpleWriteTest", context=None, llm=None):
        super().__init__(name, context, llm)

    async def run(self, context: str, k: int = 3):

        prompt = self.PROMPT_TEMPLATE.format(context=context, k=k)

        rsp = await self._aask(prompt)

        code_text = parse_code(rsp)

        return code_text

class SimpleWriteReview(Action):

    PROMPT_TEMPLATE = """
    Context: {context}
    Review the test cases and provide one critical comments:
    """

    def __init__(self, name="SimpleWriteReview", context=None, llm=None):
        super().__init__(name, context, llm)

    async def run(self, context: str):

        prompt = self.PROMPT_TEMPLATE.format(context=context)

        rsp = await self._aask(prompt)

        return rsp

#### Define Role
In many multi-agent scenarios, defining a `Role` can be as simple as 10 lines of codes. For `SimpleCoder`, we do two things:
1. Equip the `Role` with the appropriate `Action`s with `_init_actions`, this is identical to setting up a single agent
2. A multi-agent operation: we make the `Role` `_watch` important upstream messages from users or other agents. Recall our SOP, `SimpleCoder` takes user instruction, which is a `Message` caused by `BossRequirement` in MetaGPT. Therefore, we add `self._watch([BossRequirement])`.

That's all users have to do. For those who are interested in the mechanism under the hood, see Mechanism Explained of this chapter.

In [ ]:
class SimpleCoder(Role):
    def __init__(
        self,
        name: str = "Alice",
        profile: str = "SimpleCoder",
        **kwargs,
    ):
        super().__init__(name, profile, **kwargs)
        self._watch([BossRequirement])
        self._init_actions([SimpleWriteCode])

---
Similar to above, for `SimpleTester`, we:
1. Equip the `SimpleTester` with `SimpleWriteTest` action using `_init_actions`
2. Make the `Role` `_watch` important upstream messages from other agents. Recall our SOP, `SimpleTester` takes main code from `SimpleCoder`, which is a `Message` caused by `SimpleWriteCode`. Therefore, we add `self._watch([SimpleWriteCode])`.
>An extended question: Think about what it means if we use `self._watch([SimpleWriteCode, SimpleWriteReview])` instead, feel free to try this too

Additionally, we want to show that you can define your own acting logic for the agent. This applies to situation where the `Action` takes more than one input, you want to modify the input, to use particular memories, or to make any other changes to reflect specific logic. Hence, we:

3. Overwrite the `_act` function, just like what we did in a single-agent setting in [Agent101](https://colab.research.google.com/drive/1SF3bJiDjKw6Xwnz2Rf0j8Hc0U4KsSB2L#scrollTo=TJX9A8lh3FbS). Here, we want `SimpleTester` to use all memories as context for writing the test cases, and we want 5 test cases.

In [ ]:
class SimpleTester(Role):
    def __init__(
        self,
        name: str = "Bob",
        profile: str = "SimpleTester",
        **kwargs,
    ):
        super().__init__(name, profile, **kwargs)
        self._init_actions([SimpleWriteTest])
        self._watch([SimpleWriteCode])
        # self._watch([SimpleWriteCode, SimpleWriteReview]) # feel free to try this too

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self._rc.todo}")
        todo = self._rc.todo

        # context = self.get_memories(k=1)[0].content # use the most recent memory as context
        context = self.get_memories() # use all memories as context

        code_text = await todo.run(context, k=5) # specify arguments

        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg

---
Define `SimpleReviewer` following the same procedure:

In [ ]:
class SimpleReviewer(Role):
    def __init__(
        self,
        name: str = "Charlie",
        profile: str = "SimpleReviewer",
        **kwargs,
    ):
        super().__init__(name, profile, **kwargs)
        self._init_actions([SimpleWriteReview])
        self._watch([SimpleWriteTest])

### Create a team and add roles
Now that we have defined our three `Role`s, it's time to put them together. We initialize all of them, set up a `Team`, and `hire` them.

Run the `Team`, we should see the collaboration between them!

In [ ]:
async def main(
    idea: str = "write a function that calculates the product of a list",
    investment: float = 3.0,
    n_round: int = 5,
):
    logger.info(idea)

    team = Team()
    team.hire(
        [
            SimpleCoder(),
            SimpleTester(),
            SimpleReviewer(),
        ]
    )

    team.invest(investment=investment)
    team.run_project(idea)
    await team.run(n_round=n_round)

await main(idea="write a function that calculates the product of a list")

2023-11-17 03:06:22.912 | INFO     | __main__:main:6 - write a function that calculates the product of a list
2023-11-17 03:06:22.917 | INFO     | metagpt.team:invest:39 - Investment: $3.0.
2023-11-17 03:06:22.920 | INFO     | metagpt.roles.role:_act:207 - Alice(SimpleCoder): ready to SimpleWriteCode


```python
def product_of_list(lst):
    product = 1
    for num in lst:
        product *= num
    return product

# Test case 1:
print(product_of_list([1, 2, 3, 4]))  # Output: 24

# Test case 2:
print(product_of_list([5, 6, 7, 8]))  # Output: 1680
```


2023-11-17 03:06:33.123 | INFO     | metagpt.provider.openai_api:update_cost:89 - Total running cost: $0.008 | Max budget: $3.000 | Current cost: $0.008, prompt_tokens: 79, completion_tokens: 90
2023-11-17 03:06:33.129 | INFO     | __main__:_act:14 - Bob(SimpleTester): ready to SimpleWriteTest


```python
import pytest
from your_module import product_of_list

def test_product_of_list():
    assert product_of_list([1, 2, 3, 4]) == 24
    assert product_of_list([5, 6, 7, 8]) == 1680
    assert product_of_list([0, 1, 2, 3]) == 0
    assert product_of_list([10, 10, 10]) == 1000
    assert product_of_list([-1, 1, 2])

2023-11-17 03:06:45.037 | INFO     | metagpt.provider.openai_api:update_cost:89 - Total running cost: $0.020 | Max budget: $3.000 | Current cost: $0.013, prompt_tokens: 176, completion_tokens: 122
2023-11-17 03:06:45.042 | INFO     | metagpt.roles.role:_act:207 - Charlie(SimpleReviewer): ready to SimpleWriteReview


 == -2
```
The test cases seem to cover a variety of scenarios including positive numbers, zero, and negative numbers. However, one critical comment would be that there is no test case for an empty list. It would be beneficial to add a test case that handles this scenario to ensure the

2023-11-17 03:06:51.465 | INFO     | metagpt.provider.openai_api:update_cost:89 - Total running cost: $0.029 | Max budget: $3.000 | Current cost: $0.009, prompt_tokens: 170, completion_tokens: 61


 function can handle all possible inputs.
